# Init

In [1]:
import time, sys
%load_ext autoreload
%autoreload 2

## Custom tools

In [2]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
import posarmctools.readdata as readdata

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.posargb as gb

In [4]:
i_CFG = 0
i_RD = 1
i_CAL = 2
i_f = 3

i_x = 2
i_y = 3
i_z = 4

# antenna locations on the plate
Dx1   = 0.224
Dx2   = 0.168
h     = 0.04
# positions of the box in x
xMinTrail  = -1.50
xMaxTrail  = 1.50

In [5]:
from backprojection.loadbackprojection import *
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"
lib = LibBackProjection( libraryFilename )

## Numpy

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
# 'GTK', 'GTKAgg', 'GTKCairo', 'GTK3Agg', 'GTK3Cairo', 'MacOSX', 'nbAgg', 'Qt4Agg', 'Qt4Cairo', 
# 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 
# 'gdk', 'pdf', 'pgf', 'ps', 'svg', 'template'

## Data paths

In [7]:
base_path = "/home/pleroy/DATA/DIADEM/lff/Tromso_renomme"
# 1m70
base_170 = "/Kattfjord_200313/Kattfjord_X_VVVV_200313/X_VVVV_200313_1m70"
path_170 = base_path + base_170
# 1m82
base_182 = "/Kattfjord_200313/Kattfjord_X_VVVV_200313/X_VVVV_200313_1m82"
path_182 = base_path + base_182
# 2m30
base_230 = "/Kattfjord_200313/Kattfjord_X_VVVV_200313/X_VVVV_200313_2m30"
path_230 = base_path + base_230

In [8]:
prefix = base_path + "/Kattfjord_200313/Kattfjord_X_VVVV_200313/X_VVVV_200313_"
suffix = ["1m70", "1m82", "1m94", "2m06", "2m18", "2m30"]
ant = ["s21", "s31", "s32", "s41", "s42", "s43"]
Ant = ["S21", "S31", "S32", "S41", "S42", "S43"]

# Read separate files

## Read configuration

In [ ]:
CFG = gb.readCfg( path_170 + "/" +  "PoSAR.cfg")

## Read data

In [ ]:
c = gb.readFile( path_170 + "/s21.dat", CFG['Npos'], CFG['Nf'] )

In [ ]:
fix, (ax0, ax1) = plt.subplots(1,2)

im = ax0.imshow(20 * np.log10( np.abs( c ) ) )
dia.addColorBar(im, ax0)

im = ax1.imshow( np.unwrap( np.angle(c), axis = 1 ) )
dia.addColorBar(im, ax1)

In [ ]:
angle = np.angle(c[125,:])
fig, ax = plt.subplots(1,1)
ax.plot( angle, '.-' )
ax.plot( np.unwrap(angle) )
ax.grid()

## Read calibration

## Read positions

In [ ]:
ant_pos = gb.readPos( path_170 + "/track_S21.dat", CFG["Npos"] )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(ant_pos.Txx, '.-', label=f"Txx")
ax.plot(ant_pos.Txy, '.-', label=f"Txy")
ax.plot(ant_pos.Txz, '.-', label=f"Txz")
ax.plot(ant_pos.Rxx, '.-', label=f"Rxx")
ax.plot(ant_pos.Rxy, '.-', label=f"Rxy")
ax.plot(ant_pos.Rxz, '.-', label=f"Rxz")
ax.legend()
ax.grid()
title = base_170 + "\nS21"
ax.set_title(title)

# read_PoSAR_data

In [ ]:
vmax=-70
vmin=-130

## Kattfjord 1m70

In [ ]:
CFG_170, RD_170, CAL_170, ant_pos_170, f_170 = gb.readData( path_170, "s21.dat", cal_name="" )

In [ ]:
f_base = f_170 - CFG_170["Fmin"]
r = np.arange(f_base.size) * CFG_170["delta_sr"]
az = (ant_pos.Txx + ant_pos.Txz)/2

In [ ]:
hanning = np.hanning( CFG_170["Nf"] )
coupling_RD = np.average(RD_170, 0)
ifft_RD = np.fft.ifft( RD_170 )
ifft_RD_withHanning = np.fft.ifft( RD_170 * hanning )
ifft_RD_withoutCoupling = np.fft.ifft( RD_170 - coupling_RD )

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolor( r, az, 20 * np.log10( np.abs( ifft_RD_withHanning ) ), cmap='jet', vmax=vmax,vmin=vmin  )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolor( r, az, np.angle( ifft_RD_withHanning ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle(base_170 + "\ns21 with hanning")

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolor( r, az, 20 * np.log10( np.abs( ifft_RD ) ), cmap='jet' )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolor( r, az, np.angle( ifft_RD ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle(base_170 + "\ns21 raw data")

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolor( r, az, 20 * np.log10( np.abs( ifft_RD_withoutCoupling ) ), cmap='jet' )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolor( r, az, np.angle( ifft_RD_withoutCoupling ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle(base_170 + "\ns21 without coupling")

## Kattfjord 1m82

In [ ]:
CFG_182, RD_182, CAL_182, ant_pos_182, f_182 = gb.readData( path_182, "s21.dat", cal_name="" )

In [ ]:
hanning_182 = np.hanning( CFG["Nf"] )
ifft_182 = np.fft.ifft( RD_182 * hanning )

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolor( r, az, 20 * np.log10( np.abs( ifft_182 ) ), cmap='jet' )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolor( r, az, np.angle( ifft_182 ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle(base_182 + "\ns21 with hanning")

## Kattfljord 2m30

### Read data

In [ ]:
read_230 = gb.readData( prefix + "2m30", "s21.dat" )
ant_pos_230 = gb.readPos( path_230 + "/track_S21.dat", read_230[i_CFG]["Npos"] )

In [ ]:
az_230 = (ant_pos_230.Txx + ant_pos_230.Txz)/2
f_base_230 = read_230[i_f] - read_230[i_CFG]["Fmin"]
r_230 = np.arange(f_base_230.size) * read_230[i_CFG]["delta_sr"]

xyz_230      = np.zeros((read_230[i_CFG]["Npos"],5))
xyz_230[:,0] = np.arange(read_230[i_CFG]["Npos"]) # rampNumber
xyz_230[:,1] = np.zeros(read_230[i_CFG]["Npos"])  # timeStamp
xyz_230[:,2] = (ant_pos_230.Txx + ant_pos_230.Txz)/2
xyz_230[:,3] = (ant_pos_230.Rxx + ant_pos_230.Rxy)/2
xyz_230[:,4] = (ant_pos_230.Txy + ant_pos_230.Rxz)/2

### Proces data

In [ ]:
hanning_230 = np.hanning( read_230[i_CFG]["Nf"] )
ifft_230 = np.fft.ifft( read_230[i_RD] * hanning_230 )

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolor( r_230, az_230, 20 * np.log10( np.abs( ifft_230 ) ), cmap='jet', vmax=vmax,vmin=vmin )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolor( r_230, az_230, np.angle( ifft_230 ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle(base_230 + "\ns21 with hanning")

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(ant_pos_230.Txx, '.-', label=f"Txx")
ax.plot(ant_pos_230.Txy, '.-', label=f"Txy")
ax.plot(ant_pos_230.Txz, '.-', label=f"Txz")
ax.plot(ant_pos_230.Rxx, '.-', label=f"Rxx")
ax.plot(ant_pos_230.Rxy, '.-', label=f"Rxy")
ax.plot(ant_pos_230.Rxz, '.-', label=f"Rxz")
#
ax.plot(xyz_230[:,2], 'x', label=f"pos x")
ax.plot(xyz_230[:,3], 'x', label=f"pos y")
ax.plot(xyz_230[:,4], 'x', label=f"pos z")
ax.legend()
ax.grid()
title = base_230 + "\nS21"
ax.set_title(title)

### r ka

In [ ]:
r_ka_230 = np.fft.fftshift( np.fft.fft( np.fft.ifft( np.fft.ifftshift(RD,axes=0), axis=1 ), axis=0 ), axes=0 )

In [ ]:
diff_az = np.diff(az_230)
np.mean(diff_az), np.std(diff_az)
daz = 0.012

In [ ]:
ka = np.fft.fftshift(np.fft.fftfreq(read_230[0]["Npos"],daz)) * 2 * np.pi

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolor( r, ka, 20 * np.log10( np.abs( r_ka_230 ) ), cmap='jet' )
dia.addColorBar(im, ax0, 'auto')
ax0.set_xlabel('r')
ax0.set_ylabel('ka')

im = ax1.pcolor( r, ka, np.angle( r_ka_230 ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')
ax1.set_xlabel('r')
ax1.set_ylabel('ka')

fig.suptitle(base_230 + "\ns21 (r,ka)")

# Build positions

In [ ]:
zRail   = gb.getZRail( "2m30" )
xyz_dic = gb.buildPos(xMinTrail, xMaxTrail, Npos, Dx1, Dx2, h, zRail )

In [ ]:
fig, ax = plt.subplots(1,1)
for sij in ant:
    ax.plot( (xyz_dic[sij][0,i_x])+1.5, xyz_dic[sij][0,i_z], 'o', label=sij )
ax.legend()
ax.grid()
ax.set_title( f"data recorded in the files, (Dx1+2*Dx2)/2 = {offsetx}" )

In [ ]:
fig, ax = plt.subplots(1,1)
for sij in ant:
    ax.plot( (xyz_dic[sij][:,i_x]), xyz_dic[sij][:,i_z], 'o', label=sij )
ax.legend()
ax.grid()
ax.set_title("data recorded in the files")

# Define geometry

In [ ]:
hScene = -1

dx = 0.01
dy = 0.01
dz = 0.01
xMin = -1.5
xMax = 1.5
yMin = 0
yMax = 15
zMin = 2.42
zMax = 2.42+dz

x = np.arange(xMin, xMax+1e-9, dx)
y = np.arange(yMin, yMax+1e-9, dy)
z = np.arange(zMin, zMax+1e-9, dz)

nbX = x.size
nbY = y.size
nbZ = z.size

sceneX, sceneY, sceneZ = np.meshgrid(x,y,z,indexing='ij')

In [ ]:
sceneX.shape, x.shape, y.shape, z.shape

In [ ]:
print( f"center of the scene ( {np.mean(x):.2f}, {np.mean(y):.2f}, {np.mean(z):.2f} )")

# Focalization

## Backprojection

In [66]:
lib.reload()

In [49]:
Npos = gb.readCfg( prefix + "1m70/PoSAR.cfg" )["Npos"]
print( f"Npos = {Npos}" )

Npos = 250


In [50]:
read = gb.readData( prefix + "2m30", "s21.dat" )
sr = read[i_RD][:,0:read[i_CFG]["Nf"]-1]
rangeResolution = read[i_CFG]["delta_sr"]
Naz = sr.shape[0]
Nf  = sr.shape[1]

h_asString = "2m30"
zRail = gb.getZRail( h_asString )
xyz_dic = gb.buildPos(xMin, xMax, Naz, Dx1, Dx2, h, zRail )
xyzAntI = xyz_dic["1"]
xyzAntJ = xyz_dic["2"]

overSamplingRatio = 10
Nover = overSamplingRatio * Nf
r_base  = np.arange( Nf ) * rangeResolution
r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio
dr_over = r_over[1] - r_over[0]

print( f"Nf = {Nf}, Naz = {Naz}" )
print( f"range from {r_over[0]:.2f}m to {r_over[-1]:.2f}m, "
      + f"resolution = {rangeResolution:.3e}m, oversampled = {rangeResolution / overSamplingRatio:.3e}m" )

Nf = 400, Naz = 250
range from 0.00m to 14.28m, resolution = 3.571e-02m, oversampled = 3.571e-03m


In [51]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.Nover     = r_over.size
myParameters.dx        = dr_over
myParameters.Naz       = Naz
myParameters.Nf        = Nf
myParameters.phi_a_deg = 10 # 20° => 7.4cm
myParameters.kc        = read[i_CFG]["kc"]

In [52]:
t = time.time()

focusedImg_a  = np.zeros( sceneX.shape, dtype=complex )
print( "focusedImg_a.shape = {}".format( focusedImg_a.shape ) )

lib.so.backProjectionOmpGroundRange_PoSAR_GB_a( x,
                                               y,
                                               z,
                                               r_over,
                                               sr.reshape(sr.size),
                                               xyzAntI.reshape(xyzAntI.size),
                                               xyzAntJ.reshape(xyzAntJ.size),
                                               focusedImg_a.reshape(focusedImg_a.size),
                                               myParameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

focusedImg_a.shape = (3, 751, 351)
execution time = 2.8169782161712646


## Plot focused images

In [ ]:
cmap='jet'

### focusedImg_a

In [ ]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[:,:,1] ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, x, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="auto" )

In [ ]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[50,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="auto" )

### focusedImg

In [ ]:
toPlot = 20 * np.log10( np.abs( focusedImg[:,:,1] ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, x, toPlot, cmap=cmap )
dia.addColorBar( im, ax, aspect="auto" )

In [ ]:
toPlot = 20 * np.log10( np.abs( focusedImg[150,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="auto" )

### focusedImgAlt

In [ ]:
toPlot = 20 * np.log10( np.abs( focusedImgAlt[:,:,1] ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, x, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="auto" )

# Process all

## Define geometry

In [74]:
hScene = -1

dx = 0.01
dy = 0.01
dz = 0.01
xMin = -1.5
xMax = 1.5
yMin = 0
yMax = 7.5
zMin = 0
zMax = 0+dz

x = np.arange(xMin, xMax+1e-9, dx)
y = np.arange(yMin, yMax+1e-9, dy)
z = np.arange(zMin, zMax+1e-9, dz)

nbX = x.size
nbY = y.size
nbZ = z.size
centerX = int( nbX / 2 )
centerY = int( nbY / 2 )
centerZ = int( nbZ / 2 )

sceneX, sceneY, sceneZ = np.meshgrid(x,y,z,indexing='ij')

In [75]:
x.shape, y.shape, z.shape

((301,), (751,), (2,))

In [76]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.phi_a_deg = 60 # 10° => 4.3cm

## Focalization

In [77]:
suffix

['1m70', '1m82', '1m94', '2m06', '2m18', '2m30']

In [78]:
Npos = gb.readCfg( prefix + "1m70/PoSAR.cfg" )["Npos"]
print( f"Npos = {Npos}" )

Npos = 250


In [79]:
img_h = {}

d_shift = 0.2

for h_asString in suffix:
    
    data_dir = prefix + h_asString
    z1 = gb.getZRail( h_asString )
    xyz_dic = gb.buildPosUsingTrack( data_dir + "/track_S" , Dx1, Dx2, h, z1, verbose=1 )
    print( f"h_asString = {h_asString}, z1 = {z1}" )
    
    img_sij = {}
    
    for sij in ant:

        read = gb.readData( data_dir, f"{sij}.dat" )

        Rx, Tx = sij.split("s")[1][0], sij.split("s")[1][1]
        xyzRx = xyz_dic[ Rx ]
        xyzTx = xyz_dic[ Tx ]
        hAver = ( xyzRx[0,i_z] + xyzTx[0,i_z] ) / 2
    
        sr  = np.fft.ifftshift( read[i_RD][:,0:read[i_CFG]["Nf"]-1], axes=1 )
        Naz = sr.shape[0]
        Nf  = sr.shape[1]
        rangeResolution = read[i_CFG]["delta_sr"]
        overSamplingRatio = 10
        Nover   = overSamplingRatio * Nf
        r_base  = np.arange( Nf ) * rangeResolution - d_shift
        r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio - d_shift
        dr_over = r_over[1] - r_over[0]
    
        myParameters.Nover     = r_over.size
        myParameters.dx        = dr_over
        myParameters.Naz       = Naz
        myParameters.Nf        = Nf
        myParameters.kc        = read[i_CFG]["kc"]
    
        t = time.time()
    
        focusedImg  = np.zeros( sceneX.shape, dtype=complex )
        print( f"focusedImg.shape = {focusedImg.shape} *** focalization antenna {sij}, h = {hAver:.2f}"
             + f" (Rx,Tx) = ({Rx},{Tx})" )

        lib.so.backProjectionOmpGroundRange_PoSAR_GB_a( x,
                                                       y,
                                                       z,
                                                       r_over,
                                                       sr.reshape(sr.size),
                                                       xyzAntI.reshape(xyzAntI.size),
                                                       xyzAntJ.reshape(xyzAntJ.size),
                                                       focusedImg.reshape(focusedImg.size),
                                                       myParameters)

        elapsed = time.time() - t
        print("execution time = " + str(elapsed))
        img_sij[sij] = focusedImg
    img_h[h_asString] = img_sij

read 250 positions in file:
/home/pleroy/DATA/DIADEM/lff/Tromso_renomme/Kattfjord_200313/Kattfjord_X_VVVV_200313/X_VVVV_200313_1m70/track_S21.dat
h_asString = 1m70, z1 = 1.7
focusedImg.shape = (301, 751, 2) *** focalization antenna s21, h = 1.82 (Rx,Tx) = (2,1)
execution time = 1.6607589721679688
focusedImg.shape = (301, 751, 2) *** focalization antenna s31, h = 1.80 (Rx,Tx) = (3,1)
execution time = 1.6514532566070557
focusedImg.shape = (301, 751, 2) *** focalization antenna s32, h = 1.76 (Rx,Tx) = (3,2)
execution time = 1.6513762474060059
focusedImg.shape = (301, 751, 2) *** focalization antenna s41, h = 1.78 (Rx,Tx) = (4,1)
execution time = 1.6417763233184814
focusedImg.shape = (301, 751, 2) *** focalization antenna s42, h = 1.74 (Rx,Tx) = (4,2)
execution time = 1.644413948059082
focusedImg.shape = (301, 751, 2) *** focalization antenna s43, h = 1.72 (Rx,Tx) = (4,3)
execution time = 1.6442513465881348
read 250 positions in file:
/home/pleroy/DATA/DIADEM/lff/Tromso_renomme/Kattfjord_2

## Plot focused images

In [70]:
cmap='jet'

In [71]:
sij = "s21"
idxz = 300
toPlot = 20 * np.log10( np.abs( img_h["1m94"][sij][:,:,idxz]) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, x, toPlot, cmap=cmap )
dia.addColorBar( im, ax, aspect="auto" )
fig.suptitle(f"{sij} z={z[idxz]:.2f}")

Text(0.5,0.98,'s21 z=0.50')

In [57]:
sij = "s31"
idxz = 250
toPlot = 20 * np.log10( np.abs( img_h["1m94"][sij][:,:,idxz]) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, x, toPlot, cmap=cmap )
dia.addColorBar( im, ax, aspect="auto" )
fig.suptitle(f"{sij} z={z[idxz]:.2f}")

Text(0.5,0.98,'s31 z=-0.00')

In [58]:
sij = "s31"
idxz = 300
toPlot = 20 * np.log10( np.abs( img_h["1m94"][sij][:,:,idxz]) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, x, toPlot, cmap=cmap )
dia.addColorBar( im, ax, aspect="auto" )
fig.suptitle(f"{sij} z={z[idxz]:.2f}")

Text(0.5,0.98,'s31 z=0.50')

In [59]:
sij = "s31"
idxx = centerX
toPlot = 20 * np.log10( np.abs( img_h["1m94"][sij][idxx,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="auto" )
fig.suptitle(f"{sij} x={x[idxx]:.2f}")

Text(0.5,0.98,'s31 x=0.00')

In [21]:
sij = "s21"
idxx = centerX
toPlot = 20 * np.log10( np.abs( img_h["1m94"][sij][idxx,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="auto" )
fig.suptitle(f"{sij} x={x[idxx]:.2f}")

Text(0.5,0.98,'s21 x=0.00')

## sum images

### x fixed

In [68]:
idxx = centerX
sumImg = np.zeros( img_h["1m94"]["s21"][0,:,:].shape, dtype=complex )

for h_asString in suffix:
    for sij in ant:
        sumImg += img_h[h_asString][sij][ idxx, :, : ]

In [69]:
maxVal = np.amax( np.abs( sumImg.T ) )
toPlot = 20 * np.log10( np.abs( sumImg.T ) / maxVal )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap=cmap, vmin=-40, vmax=0)
dia.addColorBar( im, ax, aspect="auto" )
fig.suptitle(f"x = {x[idxx]:.2f}")

Text(0.5,0.98,'x = 0.00')

### along z